## data

- Llama 3 was **pretrained** on over 15 trillion tokens（15T） of data from **publicly available sources**.
- The **fine-tuning** data includes publicly available **instruction datasets**, as well as over **10M human-annotated examples**.
- Neither the pretraining nor the fine-tuning datasets include Meta user data.

## config

- Meta-Llama-3-8B

```
{
   "dim": 4096,
    "n_layers": 32,
    "n_heads": 32,
    "n_kv_heads": 8,
    "vocab_size": 128256,
    "multiple_of": 1024,
    "ffn_dim_multiplier": 1.3,
    "norm_eps": 1e-05,
    "rope_theta": 500000.0
}
```

- rope_theta: base
    - $5\times 10^{5}$